Министерство образования и науки Российской Федерации
Федеральное государственное бюджетное образовательное учреждение
высшего профессионального образования
«Владимирский государственный университет
имени Александра Григорьевича и Николая Григорьевича Столетовых»
(ВлГУ)

Кафедра информационных систем и программной инженерии








Лабораторная работа №3
по  дисциплине  "Теоретические основы дискретных вычислений"
Тема: "Построение минимальной ДНФ булевой функции"







Выполнил:
студент гр. ИСТ-115
Курочкин М.В.


Приняла:
Шамышева О.Н.








Владимир 2016 г.


Задание 1

Цель работы
Научиться строить минимальную дизъюнктивно нормальную форму булевой функции заданной таблицей истинности.


In [6]:
class Config:
	"""
	Класс для хранения констант

	- VARIANT: Номер варианта.
	- NUM_VARS: Количество переменных в функции.
	"""

	VARIANT = 50
	NUM_VARS = 4 # A, B, C, D

In [ ]:
import pandas as pd
from itertools import product

class LogicalFunction:
	def __init__(self):
		pass

	@staticmethod
	def variant_to_function(variant, num_vars):
		"""
        Преобразует номер варианта в инвертированное двоичное представление.
        :param variant: Номер варианта.
        :param num_vars: Количество переменных в функции.
        :return: Инвертированное двоичное представление номера варианта.
        """
		# Преобразование варианта в двоичную форму и обрезка до num_vars битов
		binary_repr = bin(variant)[2:].zfill(num_vars)[-num_vars:]
		# Инверсия каждого бита
		inverted_binary = ''.join(['1' if bit == '0' else '0' for bit in binary_repr])
		return inverted_binary

	@staticmethod
	def build_truth_table(f_binary):
		"""
		Построение таблицы истинности для заданного двоичного представления функции F.
		:param f_binary: Двоичное представление функции F.
		:return: DataFrame с таблицей истинности.
		"""
		# Использование itertools.product для генерации всех комбинаций
		combinations = product([0, 1], repeat=Config.NUM_VARS)

		# Использование генератора списков для создания таблицы истинности
		truth_table = [
			list(combination) + [1 if ''.join(map(str, combination)) == f_binary else 0]
			for combination in combinations
		]

		return pd.DataFrame(truth_table, columns=['A', 'B', 'C', 'D', 'F'])

	@staticmethod
	def find_dnf(truth_table):
		"""
		Находит ДНФ для функции на основе таблицы истинности.
		:param truth_table: DataFrame с таблицей истинности.
		:return: Строка с ДНФ.
		"""
		# Использование генератора списков для создания ДНФ
		dnf = ' | '.join([
			' & '.join([f"{var}" if row[var] else f"¬{var}" for var in ['A', 'B', 'C', 'D']])
			for _, row in truth_table.iterrows() if row['F'] == 1
		])
		return dnf


logical_function = LogicalFunction()
inverted_binary = logical_function.variant_to_function(Config.VARIANT, Config.NUM_VARS)
truth_table_df = logical_function.build_truth_table(inverted_binary)
dnf = logical_function.find_dnf(truth_table_df)
print(f'Значение F при варианте {Config.VARIANT} и количестве переменных {Config.NUM_VARS}: {inverted_binary}')
print(f'Таблица истинности')
print(truth_table_df)
print(f'ДНФ: {dnf}')


![](31.png)

Рис. 1: Результат работы программы

Выводы по 1 работе:

Определение Функции F:

Исходный вариант: 50.
Двоичное представление (4 бита): 0010.
Инвертированное двоичное представление: 1101.
Функция F определена как: F = инверсия(0010) = 1101.

Построение Таблицы Истинности:
Таблица истинности включает переменных A, B, C, D, а также их конъюнкции.

Вычисление Дизъюнктивной Нормальной Формы (ДНФ):
Поскольку F равно 1 только для комбинации 1101 (A = 1, B = 1, C = 0, D = 1), ДНФ функции F выражается как:

F=A∧B∧¬C∧D

Я освоил построение минимальной ДНФ булевой функции.
Научился использовать подходы в программировании для построения таблиц истинности на языке программирования Python.
Эти навыки расширяют мои возможности в области алгоритмического мышления, анализа логических структур и реализации сложных логических функций.

Задание 2

In [ ]:
class RecursiveTruthTableBuilder:
	"""
    Класс для построения таблицы истинности с использованием рекурсивного поразрядного сумматора.
    Он генерирует все возможные комбинации логических переменных, имена которых задаются пользователем.
    """
	def __init__(self, variables):
		"""
        Инициализирует класс с заданным списком переменных.
        :param variables: Список названий логических переменных для таблицы истинности.
        """
		self.variables = variables  # Список переменных для таблицы истинности

	def generate_combinations(self, current_vars=None, position=0):
		"""
        Рекурсивно генерирует все комбинации логических переменных.
        :param current_vars: Текущая комбинация переменных на данном этапе рекурсии.
        :param position: Текущая позиция в списке переменных.
        """
		if current_vars is None:
			current_vars = []

		if position == len(self.variables):
			self.truth_table.append(current_vars.copy())
			return

		# Для каждой переменной генерируем 0 и 1
		for val in [0, 1]:
			current_vars.append(val)
			self.generate_combinations(current_vars, position + 1)
			current_vars.pop()

	def build_table(self):
		"""
        Построение и возвращение DataFrame с таблицей истинности.
        :return: DataFrame с таблицей истинности.
        """
		self.truth_table = []  # Инициализация таблицы истинности
		self.generate_combinations()
		return pd.DataFrame(self.truth_table, columns=self.variables)


variables = ['A', 'B', 'C']
table_builder = RecursiveTruthTableBuilder(variables)
truth_table = table_builder.build_table()
truth_table




![](32.png)

Рис. 2: Результат работы программы

Выводы:
Описание Класса RecursiveTruthTableBuilder
Этот класс предназначен для генерации таблицы истинности для заданного набора логических переменных. Он позволяет задать любое количество переменных и автоматически генерирует все комбинации их значений.

Достоинства и Недостатки Использования Рекурсии
Достоинства:
Лаконичность Кода: Рекурсия позволяет сократить количество кода, делая его более читаемым и понятным.
Простота Реализации: Рекурсивные функции часто более просты в реализации для задач, связанных с перебором или итерацией по деревьям и графам.

Недостатки:
Ограничение Глубины Стека: В языках программирования, таких как Python, существует ограничение на глубину рекурсивных вызовов, что может привести к ошибке переполнения стека.
Производительность: Рекурсивные алгоритмы могут быть менее эффективными с точки зрения времени выполнения и использования памяти, особенно при большом объеме данных.

Альтернативные Методы Генерации Таблицы Истинности

Итеративные Методы: Простой цикл for или while может использоваться для генерации всех комбинаций, что устраняет риск переполнения стека.
Битовые Операции: Использование битовых операций для эффективного перебора всех возможных состояний переменных.

Задание 3

рекурсивный сумматор для системы счисления N

In [ ]:
def recursive_sum(a, b, base, carry=0):
	"""
    Рекурсивно складывает два числа в системе счисления с основанием base.
    :param a: первое число (строка).
    :param b: второе число (строка).
    :param base: основание системы счисления.
    :param carry: перенос из предыдущего разряда.
    :return: сумма в виде строки.
    """
	if not a and not b and not carry:
		return ''

	a_last, b_last = (int(a[-1]) if a else 0), (int(b[-1]) if b else 0)
	total = a_last + b_last + carry

	carry = total // base
	total %= base

	return recursive_sum(a[:-1], b[:-1], base, carry) + str(total)

a = '1234'  # Число в системе с основанием N
b = '5678'  # Ещё одно число в той же системе
base = 10   # Десятичная система счисления
result = recursive_sum(a, b, base)
print(f"Сумма чисел {a} и {b} равна {result}")

![](33.png)